# Demo Notebook Exciting Calculation
\
Show basic use cases of the exciting calculation class defined inside the ExcitingWorkflow package.\
Source code: [Gitlab ExcitingWorkflow](https://git.physik.hu-berlin.de/peschelf/excitingworkflow)\
\
Disclaimer: You need the ```excitingtools/bse_xml_generation_and_parsing``` branch from exciting for this. Will get merged soon. Groundstate only allows toplevel attributes, no subelements. Structure has no LDAplusU, dfthalfparam, shell. XS has only the here shown subelements for BSE. Talk to Fabian for more information, if you need more implemented or want to do it yourself.

In [1]:
import os
import shutil
import pathlib

from excitingworkflow.src.exciting_slurm_calculation import ExcitingCalculation
from excitingtools.input.ground_state import ExcitingGroundStateInput
from excitingtools.input.structure import ExcitingStructure
from excitingtools.input.xs import ExcitingXSInput
from excitingtools.runner import BinaryRunner

In [2]:
os.system('rm -r calculation*')  # pay attention here!!
print(os.listdir()) # no calculation directories here

['Demo_Exciting_Calculation.ipynb', '.ipynb_checkpoints']


### Exciting Groundstate calculation of LiF
Define the necessary input objects for an exciting calculation

In [3]:
# unit cell lattice of the material
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]

# Atoms inside the unit cell: every atom own dictionary with species name and position
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]

# Structure object: pass atoms, lattice, species_path and optional kwargs as dictionary
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})

# Groundstate Object with groundstate attributes
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='fromscratch',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# Exciting runner: pass binary_path, run command (serial or mpi), number of OMP threads, timeout in second
runner1 = BinaryRunner('/home/fabi/code/development/exciting/bin/exciting_smp', './', 4, 500)

# Exciting calculation: pass name, run directory, structure, groundstate, runner
gs_calculation = ExcitingCalculation('test1', 'calculation1', structure,
                                     '/home/fabi/code/exciting/species/', groundstate, runner1)

ExcitingCalculation has three main functions to use:
1. write_inputs()
2. run()
3. parse_output()

In [4]:
gs_calculation.write_inputs()

In [5]:
with open('calculation1/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test1</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate ngridk="4 4 4" rgkmax="5.0" gmaxvr="10" nempty="5" do="fromscratch" xctype="GGA_PBE_SOL" lmaxmat="12" lmaxvr="12" lmaxapw="12" epschg="0.1" epsengy="0.1" epspot="0.1"> </groundstate>
</input>




In [6]:
run_result = gs_calculation.run()

In [7]:
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time, 'in s')
print('Success:', run_result.success)

StdOut: b''
StdErr: b''
Return Code: 0
Process time: 16.10673975944519 in s
Success: True


In [8]:
result = gs_calculation.parse_output()
result

{'initialization': {'Lattice vectors (cartesian)': ['3.8040000000',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '3.8040000000',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '3.8040000000'],
  'Reciprocal lattice vectors (cartesian)': ['1.6517311533',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '1.6517311533',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '1.6517311533'],
  'Unit cell volume': '55.0454624640',
  'Brillouin zone volume': '4.5062790344',
  'Automatic determination of muffin-tin radii': '',
  'global rmt scale': '0.9500000000',
  'Species': '2 (Li)',
  'parameters loaded from': 'Li.xml',
  'name': 'lithium',
  'nuclear charge': '-3.00000000',
  'electronic charge': '3.00000000',
  'atomic mass': '12652.66897000',
  'muffin-tin radius': '1.37450000',
  '# of radial points in muffin-tin': '250',
  'atomic positions (lattice)': '',
  '1': '0.50000000  0.50000000  0.50000000',
  'Total number of atoms per unit cell':

Note: For the moment, only the `INFO.OUT` and `TOTENERGY.OUT` gets parsed. Extension is really easy (can be done by you or me, just ask me, any feedback is welcome)

In [9]:
result['initialization']['muffin-tin radius']

'1.37450000'

### Exciting BSE calculation of LiF

In [10]:
lattice = [[0.5, 0.0, 0.0], [0.0, 0.5, 0.0], [0.0, 0.0, 0.5]]
atoms = [{'species': 'Li', 'position': [0, 0, 0]},
         {'species': 'F', 'position': [0.5, 0.5, 0.5]}]
structure = ExcitingStructure(atoms, lattice, './',
                              structure_properties={'autormt': True},
                              crystal_properties={'scale': 7.608})
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='fromscratch',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# XS part definition:
# the attributes of the xs element:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                 'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}

# define the necessary sub element trees for BSE calculation
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 100}
screening_attributes = {'screentype': 'full', 'nempty': 5}
# plan_input = ['xsgeneigvec', 'writepmatxs', 'scrgeneigvec', 'scrwritepmat', 'screen', 'scrcoulint', 'exccoulint',
#              'bse']
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]

# compose the XS object with xstype and the attributes
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

runner1 = BinaryRunner('/home/fabi/code/development/exciting/bin/exciting_smp', './', 4, 500)

# BSE calculation, pass xs as last parameter
bse_calculation = ExcitingCalculation('test2', 'calculation2', structure,
                                      '/home/fabi/code/exciting/species/', groundstate, runner1, xs)

Modify only one input argument:

In [11]:
bse_calculation.xs.energywindow.attributes['points'] = '200'  # has to be passed as str

In [12]:
bse_calculation.write_inputs()
with open('calculation2/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test2</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate ngridk="4 4 4" rgkmax="5.0" gmaxvr="10" nempty="5" do="fromscratch" xctype="GGA_PBE_SOL" lmaxmat="12" lmaxvr="12" lmaxapw="12" epschg="0.1" epsengy="0.1" epspot="0.1"> </groundstate>
	<xs xstype="BSE" broad="0.327" ngridk="1 1 1" nempty="5" ngridq="1 1 1" gqmax="2.5" tappinfo="true" tevout="true" vkloff="0.05 0.03 0.13">
		<screening screentype="full" nempty="5"> </screening>
		<BSE bsetype="IP" xas="true" nstlxas="1 5" xasatom="1" xasedge="K" xasspecies="1"> </BSE>
		<energywindow intv="50 80" points="200"> </energywindow>
		<qpointset>
			<qpoint>0 0 0</qpoi

In [13]:
run_result = bse_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b'\rSolving BSE Eigenvalue Problem:                     100.000\n'
StdErr: b''
Return Code: 0
Process time: 25.226682424545288
Success: True


In [14]:
result_2 = bse_calculation.parse_output()
print(result_2)

{'frequency': array([1360.569193  , 1364.65090058, 1368.73260816, 1372.81431574,
       1376.89602332, 1380.97773089, 1385.05943847, 1389.14114605,
       1393.22285363, 1397.30456121, 1401.38626879, 1405.46797637,
       1409.54968395, 1413.63139153, 1417.71309911, 1421.79480668,
       1425.87651426, 1429.95822184, 1434.03992942, 1438.121637  ,
       1442.20334458, 1446.28505216, 1450.36675974, 1454.44846732,
       1458.5301749 , 1462.61188247, 1466.69359005, 1470.77529763,
       1474.85700521, 1478.93871279, 1483.02042037, 1487.10212795,
       1491.18383553, 1495.26554311, 1499.34725069, 1503.42895826,
       1507.51066584, 1511.59237342, 1515.674081  , 1519.75578858,
       1523.83749616, 1527.91920374, 1532.00091132, 1536.0826189 ,
       1540.16432648, 1544.24603406, 1548.32774163, 1552.40944921,
       1556.49115679, 1560.57286437, 1564.65457195, 1568.73627953,
       1572.81798711, 1576.89969469, 1580.98140227, 1585.06310985,
       1589.14481742, 1593.226525  , 1597.308232

### Exciting BSE calculation on top of groundstate Calculation

In [15]:
xs_attributes = {'broad': 0.327, 'ngridk': [1, 1, 1], 'nempty': 5, 'ngridq': [1, 1, 1], 'gqmax': 2.5,
                     'tappinfo': True, 'tevout': True, 'vkloff': [0.05, 0.03, 0.13]}
bse_attributes = {'bsetype': 'IP', 'xas': True, 'nstlxas': [1, 5], 'xasatom': 1, 'xasedge': 'K',
                  'xasspecies': '1'}
energywindow_attributes = {'intv': [50, 80], 'points': 100}
screening_attributes = {'screentype': 'full', 'nempty': 5}
plan_input = ['xsgeneigvec', 'writepmatxs', 'bse']
qpointset_input = [[0, 0, 0]]
xs = ExcitingXSInput("BSE", xs=xs_attributes,
                     BSE=bse_attributes,
                     energywindow=energywindow_attributes,
                     screening=screening_attributes,
                     qpointset=qpointset_input,
                     plan=plan_input)

runner1 = BinaryRunner('/home/fabi/code/development/exciting/bin/exciting_smp', './', 4, 500)

pure_bse_calculation = ExcitingCalculation('test3', 'calculation3', 'calculation1', 'calculation1',
                                           'calculation1', runner1, xs)

In [16]:
pure_bse_calculation.write_inputs()
with open('calculation3/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test3</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0.0 0.0 0.0"> </atom>
		</species>
	</structure>
	<groundstate ngridk="4 4 4" rgkmax="5.0" gmaxvr="10" nempty="5" do="skip" xctype="GGA_PBE_SOL" lmaxmat="12" lmaxvr="12" lmaxapw="12" epschg="0.1" epsengy="0.1" epspot="0.1"> </groundstate>
	<xs xstype="BSE" broad="0.327" ngridk="1 1 1" nempty="5" ngridq="1 1 1" gqmax="2.5" tappinfo="true" tevout="true" vkloff="0.05 0.03 0.13">
		<screening screentype="full" nempty="5"> </screening>
		<BSE bsetype="IP" xas="true" nstlxas="1 5" xasatom="1" xasedge="K" xasspecies="1"> </BSE>
		<energywindow intv="50 80" points="100"> </energywindow>
		<qpointset>
			<qpoint>0 0 0</qpoin

In [17]:
os.listdir('calculation3')

['EFERMI.OUT', 'Li.xml', 'STATE.OUT', 'input.xml', 'F.xml']

In [18]:
run_result = pure_bse_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b'\rSolving BSE Eigenvalue Problem:                     100.000\n'
StdErr: b''
Return Code: 0
Process time: 9.866629838943481
Success: True


In [19]:
result_3 = pure_bse_calculation.parse_output()
result_3

{'frequency': array([1360.569193  , 1368.73260816, 1376.89602332, 1385.05943847,
        1393.22285363, 1401.38626879, 1409.54968395, 1417.71309911,
        1425.87651426, 1434.03992942, 1442.20334458, 1450.36675974,
        1458.5301749 , 1466.69359005, 1474.85700521, 1483.02042037,
        1491.18383553, 1499.34725069, 1507.51066584, 1515.674081  ,
        1523.83749616, 1532.00091132, 1540.16432648, 1548.32774163,
        1556.49115679, 1564.65457195, 1572.81798711, 1580.98140227,
        1589.14481742, 1597.30823258, 1605.47164774, 1613.6350629 ,
        1621.79847806, 1629.96189321, 1638.12530837, 1646.28872353,
        1654.45213869, 1662.61555385, 1670.778969  , 1678.94238416,
        1687.10579932, 1695.26921448, 1703.43262964, 1711.59604479,
        1719.75945995, 1727.92287511, 1736.08629027, 1744.24970543,
        1752.41312058, 1760.57653574, 1768.7399509 , 1776.90336606,
        1785.06678122, 1793.23019637, 1801.39361153, 1809.55702669,
        1817.72044185, 1825.8838570

In [20]:
result_2['imag_oscillator_strength'].all() == result_3['imag_oscillator_strength'].all()

True

### Using the groundstate calculation object
Note: You dont have to specify all objects again, as jupyter stores them

In [21]:
pure_bse_calculation2 = ExcitingCalculation('test4', 'calculation4', gs_calculation, gs_calculation, 
                                            gs_calculation, runner1, xs)

In [22]:
pure_bse_calculation2.write_inputs()
with open('calculation4/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test4</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0 0 0"> </atom>
		</species>
	</structure>
	<groundstate ngridk="4 4 4" rgkmax="5.0" gmaxvr="10" nempty="5" do="skip" xctype="GGA_PBE_SOL" lmaxmat="12" lmaxvr="12" lmaxapw="12" epschg="0.1" epsengy="0.1" epspot="0.1"> </groundstate>
	<xs xstype="BSE" broad="0.327" ngridk="1 1 1" nempty="5" ngridq="1 1 1" gqmax="2.5" tappinfo="true" tevout="true" vkloff="0.05 0.03 0.13">
		<screening screentype="full" nempty="5"> </screening>
		<BSE bsetype="IP" xas="true" nstlxas="1 5" xasatom="1" xasedge="K" xasspecies="1"> </BSE>
		<energywindow intv="50 80" points="100"> </energywindow>
		<qpointset>
			<qpoint>0 0 0</qpoint>
		<

In [23]:
run_result = pure_bse_calculation2.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b'\rSolving BSE Eigenvalue Problem:                     100.000\n'
StdErr: b''
Return Code: 0
Process time: 9.808221578598022
Success: True


In [24]:
result_4 = pure_bse_calculation2.parse_output()
result_4

{'frequency': array([1360.569193  , 1368.73260816, 1376.89602332, 1385.05943847,
        1393.22285363, 1401.38626879, 1409.54968395, 1417.71309911,
        1425.87651426, 1434.03992942, 1442.20334458, 1450.36675974,
        1458.5301749 , 1466.69359005, 1474.85700521, 1483.02042037,
        1491.18383553, 1499.34725069, 1507.51066584, 1515.674081  ,
        1523.83749616, 1532.00091132, 1540.16432648, 1548.32774163,
        1556.49115679, 1564.65457195, 1572.81798711, 1580.98140227,
        1589.14481742, 1597.30823258, 1605.47164774, 1613.6350629 ,
        1621.79847806, 1629.96189321, 1638.12530837, 1646.28872353,
        1654.45213869, 1662.61555385, 1670.778969  , 1678.94238416,
        1687.10579932, 1695.26921448, 1703.43262964, 1711.59604479,
        1719.75945995, 1727.92287511, 1736.08629027, 1744.24970543,
        1752.41312058, 1760.57653574, 1768.7399509 , 1776.90336606,
        1785.06678122, 1793.23019637, 1801.39361153, 1809.55702669,
        1817.72044185, 1825.8838570

In [25]:
result_4['imag_oscillator_strength'].all() == result_3['imag_oscillator_strength'].all()

True

### Error:

In [26]:
# Groundstate Object with groundstate attributes
groundstate = ExcitingGroundStateInput(ngridk=[4, 4, 4], rgkmax=5.0, gmaxvr=10, nempty=5, do='keks',
                                       xctype='GGA_PBE_SOL', lmaxmat=12, lmaxvr=12, lmaxapw=12, epschg=0.1,
                                       epsengy=0.1, epspot=0.1)

# Exciting calculation: pass name, run directory, structure, groundstate, runner
error_calculation = ExcitingCalculation('test5', 'calculation5', structure,
                                   '/home/fabi/code/exciting/species/', groundstate, runner1)

In [27]:
error_calculation.write_inputs()
run_result = error_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b" Parser ERROR: 'keks                                                                            ' is not valid selection for do \n"
StdErr: b''
Return Code: 0
Process time: 0.0034198760986328125
Success: True


Error handling in exciting not existing... yet!

### Parse input.xml and modify just slightly:

In [28]:
from excitingtools.parser.input_parser import parse_input_xml

In [29]:
input_dict = parse_input_xml('calculation1/input.xml')
input_dict

{'structure': <excitingtools.input.structure.ExcitingStructure at 0x7f4d380d3a60>,
 'ground_state': <excitingtools.input.ground_state.ExcitingGroundStateInput at 0x7f4d63ad80a0>}

In [30]:
structure = input_dict['structure']
ground_state = input_dict['ground_state']
ground_state.attributes

{'ngridk': '4 4 4',
 'rgkmax': '5.0',
 'gmaxvr': '10',
 'nempty': '5',
 'do': 'fromscratch',
 'xctype': 'GGA_PBE_SOL',
 'lmaxmat': '12',
 'lmaxvr': '12',
 'lmaxapw': '12',
 'epschg': '0.1',
 'epsengy': '0.1',
 'epspot': '0.1'}

In [31]:
ground_state.attributes['ngridk'] = '5 5 5'  # as string again

In [32]:
new_calculation = ExcitingCalculation('test6', 'calculation6', structure,
                                   '/home/fabi/code/exciting/species/', ground_state, runner1)

In [33]:
new_calculation.write_inputs()
with open('calculation6/input.xml', 'r') as fid:
    print(fid.read())

<?xml version="1.0" ?>
<input>
	<title>test6</title>
	<structure speciespath="./" autormt="true">
		<crystal scale="7.608">
			<basevect>0.5 0.0 0.0</basevect>
			<basevect>0.0 0.5 0.0</basevect>
			<basevect>0.0 0.0 0.5</basevect>
		</crystal>
		<species speciesfile="F.xml">
			<atom coord="0.5 0.5 0.5"> </atom>
		</species>
		<species speciesfile="Li.xml">
			<atom coord="0.0 0.0 0.0"> </atom>
		</species>
	</structure>
	<groundstate ngridk="5 5 5" rgkmax="5.0" gmaxvr="10" nempty="5" do="fromscratch" xctype="GGA_PBE_SOL" lmaxmat="12" lmaxvr="12" lmaxapw="12" epschg="0.1" epsengy="0.1" epspot="0.1"> </groundstate>
</input>




In [34]:
run_result = new_calculation.run()
print('StdOut:', run_result.stdout)
print('StdErr:', run_result.stderr)
print('Return Code:', run_result.return_code)
print('Process time:', run_result.process_time)
print('Success:', run_result.success)

StdOut: b''
StdErr: b''
Return Code: 0
Process time: 14.942440748214722
Success: True


In [35]:
result_5 = new_calculation.parse_output()
result_5

{'initialization': {'Lattice vectors (cartesian)': ['3.8040000000',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '3.8040000000',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '3.8040000000'],
  'Reciprocal lattice vectors (cartesian)': ['1.6517311533',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '1.6517311533',
   '0.0000000000',
   '0.0000000000',
   '0.0000000000',
   '1.6517311533'],
  'Unit cell volume': '55.0454624640',
  'Brillouin zone volume': '4.5062790344',
  'Automatic determination of muffin-tin radii': '',
  'global rmt scale': '0.9500000000',
  'Species': '2 (Li)',
  'parameters loaded from': 'Li.xml',
  'name': 'lithium',
  'nuclear charge': '-3.00000000',
  'electronic charge': '3.00000000',
  'atomic mass': '12652.66897000',
  'muffin-tin radius': '1.37450000',
  '# of radial points in muffin-tin': '250',
  'atomic positions (lattice)': '',
  '1': '0.50000000  0.50000000  0.50000000',
  'Total number of atoms per unit cell':